# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 我不流行二十年, ID: 10614
Movie: 下女的誘惑, ID: 10610
Movie: 樂來越愛你, ID: 10609
Movie: 謝謝你, ID: 10571
Movie: 搞鬼, ID: 10570
Movie: 政客誠實中, ID: 10563
Movie: 女聲我最美, ID: 10528
Movie: 真實, ID: 10523
Movie: 特別嘉賓, ID: 10497
Movie: 來自深淵 深沉靈魂的黎明, ID: 10483
Movie: 霸王別姬, ID: 10594
Movie: 阿飛正傳, ID: 10593
Movie: 在黑暗中漫舞：20週年數位修復版, ID: 10590
Movie: 拾光人生, ID: 10581
Movie: 羊男的迷宮, ID: 10580
Movie: 冤鴦大盜, ID: 10573
Movie: 馬丁伊登, ID: 10569
Movie: 火口的二人, ID: 10555
Movie: 犬鳴村, ID: 10543
Movie: 間諜速成班, ID: 10538
Movie: 我的英雄學院劇場版：英雄新世紀, ID: 10481
Movie: 魔髮精靈唱遊世界, ID: 10466
Movie: 小魔花, ID: 10068
Movie: 我的學生都是配樂大師, ID: 10557
Movie: 我們的家不是我們的家, ID: 10556
Movie: 聲入奇境：經典配樂大師, ID: 10550
Movie: 絕命直播, ID: 10548
Movie: 託陰2：布拉姆回來了, ID: 10540
Movie: 迷雁返家路, ID: 10533
Movie: 無聲救援, ID: 10521
Movie: 鏡中驚魂, ID: 10514
Movie: 最初的晚餐, ID: 10510
Movie: 抓住救命稻草的野獸們, ID: 10507
Movie: 第六感生死戀, ID: 10504
Movie: Re從零開始的異世界生活 外傳集, ID: 10480
Movie: 記憶屋, ID: 10470
Movie: 不愛鋼琴師, ID: 10542
Movie: 神之一手：鬼手篇, ID: 10520
Movie: A+瞎妹, ID: 10508
Movie: 搖滾師匠：細野

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [3]:
soup

<!DOCTYPE html>
<html lang="zh_TW">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="501887343352051" property="fb:app_id"/>
<meta content="Yahoo奇摩電影" property="og:site_name"/>
<title>Yahoo奇摩電影</title>
<meta content="Yahoo奇摩電影" name="twitter:title"/>
<meta content="Yahoo奇摩電影" property="og:title"/>
<link href="https://movies.yahoo.com.tw" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw" property="og:url"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="twitter:description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" property="og:description"/>
<meta content="https://movies.yahoo.com.tw/build/images/ogImage.png" name="twitter:image"/>
<meta content="https://movies.ya

In [4]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10483

In [5]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 花蓮, 代號(area_id): 12


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 28

In [7]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

四月 14
四月 15
四月 16
四月 17
四月 18


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [8]:
# 選定要觀看的日期
date = "2020-04-15"

In [30]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'area_id':str(area_id),
              'date':date,
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
print('='*50)
for i in html:
    print('電影院: ',i.find('a').text)
    for j in i.find_all('input'):
        print("放映類型: %s \n%s "%(j['data-movie_type'],j['value']))
    print('='*50)
    

電影院:  台北日新威秀影城
放映類型: 數位 
2020-04-15 14:05:00 
放映類型: 數位 
2020-04-15 16:20:00 
放映類型: 數位 
2020-04-15 18:35:00 
電影院:  台北信義威秀影城
放映類型: 數位 
2020-04-15 11:00:00 
放映類型: 數位 
2020-04-15 13:20:00 
放映類型: 數位 
2020-04-15 15:40:00 
放映類型: 數位 
2020-04-15 18:00:00 
放映類型: 數位 
2020-04-15 20:20:00 
放映類型: 數位 
2020-04-15 22:40:00 
電影院:  京站威秀影城
放映類型: 數位 
2020-04-15 12:10:00 
放映類型: 數位 
2020-04-15 14:25:00 
放映類型: 數位 
2020-04-15 16:40:00 
放映類型: 數位 
2020-04-15 19:05:00 
電影院:  MUVIE CINEMAS台北松仁威秀
放映類型: 數位 
2020-04-15 16:00:00 
放映類型: 數位 
2020-04-15 20:15:00 
